In [ ]:
import sys
from arnie.mfe import mfe
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
nan = np.nan

In [ ]:
#Replace '-999'in shape data to nan 
def shape_lines_unnorm(shape_lines):
    all_shape_lines_no_nan = []
    for number in shape_lines:
        if number == '-999\n':
            all_shape_lines_no_nan.append('nan')
        elif number =='nan\n':
             all_shape_lines_no_nan.append('nan')
        elif number =='nan': #line added for 'nan' in dms shape
             all_shape_lines_no_nan.append('nan')
        else:
            all_shape_lines_no_nan.append(number)   
    float_lines=[float(i) for i in all_shape_lines_no_nan]
    return (float_lines)

In [ ]:
#Normalizzing the shape data
def normalize_shape(shape_reacs):
    shape_reacs = np.array(shape_reacs)

    # Get rid of nan values for now
    nonan_shape_reacs = shape_reacs[~np.isnan(shape_reacs)]

    # Find Filter 1: 1.5 * Inter-Quartile Range
    sorted_shape = np.sort(nonan_shape_reacs)
    q1 = sorted_shape[int(0.25 * len(sorted_shape))]
    q3 = sorted_shape[int(0.75 * len(sorted_shape))]
    iq_range = abs(q3 - q1)
    filter1 = next(x for x, val in \
        enumerate(list(sorted_shape)) if val > 1.5 * iq_range)

    # Find Filter 2: 95% value
    filter2 = int(0.95 * len(sorted_shape))

    # Get maximum filter value and fiter data
    filter_cutoff = sorted_shape[max(filter1, filter2)]
    sorted_shape = sorted_shape[sorted_shape < filter_cutoff]

    # Scalefactor: Mean of top 10th percentile of values
    top90 = sorted_shape[int(0.9 * len(sorted_shape))]
    scalefactor = np.mean(sorted_shape[sorted_shape > top90])
        
    # Scale dataset
    return shape_reacs/scalefactor

In [ ]:
#Possible normalized shape values 0-2
def shape_norm_for_heatmap(shape_norm):
    all_shape_lines_no_nan = []
    for number in shape_norm:
        if number < 0:
            all_shape_lines_no_nan.append(0)
        elif number > 2:
            all_shape_lines_no_nan.append(2)
        else:
            all_shape_lines_no_nan.append(number)   
    float_lines=[float(i) for i in all_shape_lines_no_nan]
    return (float_lines)

In [ ]:
#Half the shape values
def divide_by_2(shape_lines):
    quotients = []
    for number in shape_lines:
        quotients.append(number / 2)
    return quotients

In [ ]:
#Nest function for inputting the shape_lines
def function_for_heatmap_data(shape_lines):
    shape_lines_unnormalize = shape_lines_unnorm(shape_lines) 
    shape_lines_norm = normalize_shape(shape_lines_unnormalize)
    shape_norm_for_heatmap_var = shape_norm_for_heatmap(shape_lines_norm)
    shape_output = divide_by_2(shape_norm_for_heatmap_var)
    return shape_output

In [ ]:
def new_import(file_name):
    shape_fname = file_name 
    f = open(shape_fname,encoding='utf-8-sig')
    shape_lines= f.readlines()
    f.close()
    return shape_lines

In [ ]:
#Changing "T" to "U" in sequence 
def sequence_no_T(sequence):
    reactivity_no_GU= []
    for idx in range(0, len(sequence)):
        if sequence[idx] =='T':
            reactivity_no_GU.append('U')
        else:
            reactivity_no_GU.append(sequence[idx])
    return reactivity_no_GU

In [ ]:
# Replacing dot bracket to binary notation
def bp_func(dot_bracket):
    struct_sim= []
    for idx in range(0, len(dot_bracket)):
        if dot_bracket[idx] == ".":
            struct_sim.append(1) #1 means base is unpaired
        else:
            struct_sim.append(0) #0 means base is paired
    return(struct_sim)

In [ ]:
#Half the shape values
def divide_by_2(shape_lines):
    quotients = []
    for number in shape_lines:
        quotients.append(number / 2)
    return quotients

In [ ]:
#SARS-Cov-2 whole genome sequence
f = open('refseq.txt')
whole_genome= f.readlines()[0].strip('\n').replace('T','U')
f.close()


In [ ]:
#Import shape data from csv files
shape_lines_incarnato = new_import('incarnato_normalized_reactivity.csv')
shape_lines_pyle = new_import('pyle_normalized_reactivity.csv')
shape_lines_zhang_invivo = new_import('zhang_invivo_normalized_reactivity.csv')
shape_lines_zhang_invitro = new_import('zhang_invitro_normalized_reactivity.csv') 

In [ ]:
shape_norm_for_heatmap_incarnato_whole_genome =function_for_heatmap_data(shape_lines_incarnato)

shape_norm_for_heatmap_pyle_whole_genome =function_for_heatmap_data(shape_lines_pyle)

shape_norm_for_heatmap_zhang_invivo_whole_genome =function_for_heatmap_data(shape_lines_zhang_invivo)

shape_norm_for_heatmap_zhang_invitro_whole_genome = function_for_heatmap_data(shape_lines_zhang_invitro)

In [ ]:
#xaxis
position_whole_genome=list(range(1,29904))
xaxis_whole_genome = [''.join([whole_genome[i], str(position_whole_genome[i])]) for i in range(0, len(position_whole_genome))]

In [ ]:
def covert_conservation_to_floats(conservation_lines):
    conservation_numbers = []
    for number in conservation_lines:
        if number == '\ufeff1\n':
            conservation_numbers.append(1)
        else:
            conservation_numbers.append(number)   
    float_lines=[float(i) for i in conservation_numbers]
    return (float_lines)

In [ ]:
#Conservation track
sars_conservation_lines = new_import('sarsr_conservation.csv') 
conservation_floats = covert_conservation_to_floats(sars_conservation_lines)
conservation_whole_genome = conservation_floats


In [ ]:
#Datasets in 100 nucleotide windows
shape_norm_for_heatmap_pyle_whole_genome_frag =([shape_norm_for_heatmap_incarnato_whole_genome[i:i+100] for i in range(0, len(shape_norm_for_heatmap_incarnato_whole_genome), 100)])

shape_norm_for_heatmap_incarnato_whole_genome_frag =([shape_norm_for_heatmap_pyle_whole_genome[i:i+100] for i in range(0, len(shape_norm_for_heatmap_pyle_whole_genome), 100)])

shape_norm_for_heatmap_zhang_invivo_whole_genome_frag =([shape_norm_for_heatmap_zhang_invivo_whole_genome[i:i+100] for i in range(0, len(shape_norm_for_heatmap_zhang_invivo_whole_genome), 100)])

shape_norm_for_heatmap_zhang_invitro_whole_genome_frag =([shape_norm_for_heatmap_zhang_invitro_whole_genome[i:i+100] for i in range(0, len(shape_norm_for_heatmap_zhang_invitro_whole_genome), 100)])

In [ ]:
#supplementary data in 100 nucleotide windows
xaxis_whole_genome_frag =([xaxis_whole_genome[i:i+100] for i in range(0, len(xaxis_whole_genome), 100)])
conservation_whole_genome_frag =([conservation_whole_genome[i:i+100] for i in range(0, len(conservation_whole_genome), 100)])

In [ ]:
#Import structured prediction data; for i in data: [start nucleotide, end nucleotide, sequence, dot bracket, nan]
incarnato_predicted_structure_lines = new_import('incarnato_secondary_structures.csv')
pyle_predicted_structure_lines = new_import('pyle_predicted_structures.csv')
rnaz_predicted_structure_lines = new_import('rnaz_structured_predictions.csv')
zhang_predicted_structure_lines = new_import('zhang_structures.csv')

In [ ]:
incarnato_start=([incarnato_predicted_structure_lines[i].split(',')[0] for i in range(0, len(incarnato_predicted_structure_lines))])
pyle_start=([pyle_predicted_structure_lines[i].split(',')[0] for i in range(0, len(pyle_predicted_structure_lines))])
rnaz_start=([rnaz_predicted_structure_lines[i].split(',')[0] for i in range(0, len(rnaz_predicted_structure_lines))])
zhang_start=([zhang_predicted_structure_lines[i].split(',')[0] for i in range(0, len(zhang_predicted_structure_lines))])


In [ ]:
incarnato_end=([incarnato_predicted_structure_lines[i].split(',')[1] for i in range(0, len(incarnato_predicted_structure_lines))])
pyle_end=([pyle_predicted_structure_lines[i].split(',')[1] for i in range(0, len(pyle_predicted_structure_lines))])
rnaz_end=([rnaz_predicted_structure_lines[i].split(',')[1] for i in range(0, len(rnaz_predicted_structure_lines))])
zhang_end=([zhang_predicted_structure_lines[i].split(',')[1] for i in range(0, len(zhang_predicted_structure_lines))])

In [ ]:
incarnato_struct=([incarnato_predicted_structure_lines[i].split(',')[3] for i in range(0, len(incarnato_predicted_structure_lines))])
pyle_struct=([pyle_predicted_structure_lines[i].split(',')[3] for i in range(0, len(pyle_predicted_structure_lines))])
rnaz_struct=([rnaz_predicted_structure_lines[i].split(',')[3] for i in range(0, len(rnaz_predicted_structure_lines))])
zhang_struct=([zhang_predicted_structure_lines[i].split(',')[3] for i in range(0, len(zhang_predicted_structure_lines))])


In [ ]:
#Convert dot bracket to binary
incarnato_struct_dot=([bp_func(incarnato_struct[i]) for i in range(0, len(incarnato_predicted_structure_lines))])
pyle_struct_dot=([bp_func(pyle_struct[i]) for i in range(0, len(pyle_predicted_structure_lines))])
rnaz_struct_dot=([bp_func(rnaz_struct[i]) for i in range(0, len(rnaz_predicted_structure_lines))])
zhang_struct_dot=([bp_func(zhang_struct[i]) for i in range(0, len(zhang_predicted_structure_lines))])
zhang_struct_dot=([bp_func(zhang_struct[i]) for i in range(0, len(zhang_predicted_structure_lines))])

In [ ]:
def heatmap_secondary_struct_incarnato(start,end,dot_bracket):    
    whole_genome_number = list(range(1,29904))
    for i in range(len(start)):
                   start_ind = int(start[i])
                   end_ind = int(end[i])-1 # index
                   ind = 0
                   curr = dot_bracket[i]
                   for j in range(start_ind,end_ind):
                       whole_genome_number[j]= curr[ind]
                       ind+=1
    return whole_genome_number

In [ ]:
def heatmap_secondary_struct(start,end,dot_bracket):    
    whole_genome_number = list(range(1,29904))
    for i in range(len(start)):
                   start_ind = int(start[i])-1
                   end_ind = int(end[i])
                   ind = 0
                   curr = dot_bracket[i]
                   for j in range(start_ind,end_ind):
                       whole_genome_number[j]= curr[ind]
                       ind+=1
    return whole_genome_number

In [ ]:
def new_dot_func(dot_bracket):
    struct_sim= []
    for idx in range(0, len(dot_bracket)):
        if dot_bracket[idx] == 1:
            struct_sim.append(1) #1 means base is unpaired
        elif dot_bracket[idx] == 0:
            struct_sim.append(0)
        else:
            struct_sim.append(nan) #0 means base is paired
    return(struct_sim)

In [ ]:
predicted_sec_struct_incarnato = heatmap_secondary_struct_incarnato(incarnato_start,incarnato_end,incarnato_struct_dot)
new_incarnato_struc = new_dot_func(predicted_sec_struct_incarnato)
new_incarnato_struc[0]=nan

In [ ]:
predicted_sec_struct_pyle = heatmap_secondary_struct(pyle_start, pyle_end, pyle_struct_dot)
new_pyle_struc = new_dot_func(predicted_sec_struct_pyle)


In [ ]:
predicted_sec_struct_rnaz = heatmap_secondary_struct(rnaz_start,rnaz_end,rnaz_struct_dot)
new_rnaz_struc = new_dot_func(predicted_sec_struct_rnaz)

In [ ]:
predicted_sec_struct_zhang = heatmap_secondary_struct(zhang_start,zhang_end,zhang_struct_dot)
new_zhang_struc = new_dot_func(predicted_sec_struct_zhang)
new_zhang_struc[0]=nan


In [ ]:
#100 nucleotide windows for new tracks
incarnato_struct_track =([new_incarnato_struc[i:i+100] for i in range(0, len(new_incarnato_struc), 100)])
pyle_struct_track =([new_pyle_struc[i:i+100] for i in range(0, len(new_pyle_struc), 100)])
rnaz_struct_track =([new_rnaz_struc[i:i+100] for i in range(0, len(new_rnaz_struc), 100)])
zhang_struct_track =([new_zhang_struc[i:i+100] for i in range(0, len(new_zhang_struc), 100)])

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
cdict = {'red': [[0, 1, 1], [0.1, 1, 1], [0.8, 0.96, 0.96], [1.0, 0.9666, 0.9666]],
        'green': [[0, 1, 1], [0.1, 1, 1], [0.8, 0.71, 0.71], [1.0, 0.59, 0.59]], 
        'blue': [[0, 1, 1], [0.1, 1, 1], [0.8, 0.42, 0.42], [1.0, 0.35, 0.35]]}
color_map = LinearSegmentedColormap('testCmap', segmentdata=cdict, N=256)
color_map_red = plt.cm.get_cmap('Reds') # Set up color scheme
color_map.set_bad("gainsboro") # nan values will now be gray

In [ ]:
def conserv_track(name,xaxis, i):
    data= np.array([name]) 
    data_df = pd.DataFrame(data, \
                index=["conservation"], \
                columns=xaxis)
    heatmap = sns.heatmap(data_df, cmap='Greys', ax=ax[i], xticklabels=False)
    for i in range(data_df.shape[0] + 1):
        ax[0].axhline(i, color='white', lw=4)
    heatmap.set_yticklabels(heatmap.get_yticklabels(),va='center', rotation=0)    


In [ ]:
def struct_track(data_a, data_b, data_c, data_d,xaxis,i):
    data= np.array([data_a, data_b, data_c, data_d]) 
    data_df = pd.DataFrame(data, \
                index=["Manfredonia, et al. structure","Huston, et al. structure", "Rangan, et al. structure", "Sun, et al. structure"], \
                columns=xaxis)
    heatmap_a = sns.heatmap(data_df, cmap=color_map, ax=ax[i], xticklabels=False)
    for i in range(data_df.shape[0] + 1):
        ax[1].axhline(i, color='white', lw=4)
    heatmap_a.set_yticklabels(heatmap_a.get_yticklabels(),va='center', rotation=0)


In [ ]:
def shape_track(data_a, data_b, data_c, data_d,xaxis,i):   
    data = np.array([data_a, data_b, data_c, data_d]) 
    data_df= pd.DataFrame(data, \
            index=["Manfredonia, et al. NAR 2020", "Sun, et al. invitro Cell 2021", "Sun, et al. invivo Cell 2021", "Huston, et al. Mol. Cell 2021)"], \
            columns=xaxis)
    mask = data_df.isnull() # Create a mask for all the "nan" values
    heatmap = sns.heatmap(data_df, mask=mask, cmap=color_map, ax=ax[i])
    heatmap.set_yticklabels(heatmap.get_yticklabels(),va='center', rotation=0)

In [ ]:
#heatmap for whole genome

import matplotlib as plt
plt.rcParams.update({'figure.max_open_warning': 0})

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
pp = PdfPages('genome_wide_structure_comparison.pdf')
sns.set(font_scale = 4)

for idx in range(0, 300, 3):  
    data1a = shape_norm_for_heatmap_incarnato_whole_genome_frag[idx]
    data2a = shape_norm_for_heatmap_zhang_invitro_whole_genome_frag[idx]
    data3a = shape_norm_for_heatmap_zhang_invivo_whole_genome_frag[idx]
    data4a = shape_norm_for_heatmap_pyle_whole_genome_frag[idx]

    
    xaxisa = xaxis_whole_genome_frag[idx]
    conservationa = conservation_whole_genome_frag[idx]
    struct1a = incarnato_struct_track[idx]
    struct2a = pyle_struct_track[idx]
    struct3a = rnaz_struct_track[idx]
    struct4a= zhang_struct_track[idx]
    
    data1b = shape_norm_for_heatmap_incarnato_whole_genome_frag[idx+1]
    data2b = shape_norm_for_heatmap_zhang_invitro_whole_genome_frag[idx+1]
    data3b = shape_norm_for_heatmap_zhang_invivo_whole_genome_frag[idx+1]
    data4b = shape_norm_for_heatmap_pyle_whole_genome_frag[idx+1]
    
    xaxisb = xaxis_whole_genome_frag[idx+1]
    conservationb = conservation_whole_genome_frag[idx+1]
    struct1b = incarnato_struct_track[idx+1]
    struct2b = pyle_struct_track[idx+1]
    struct3b = rnaz_struct_track[idx+1]
    struct4b = zhang_struct_track[idx+1]

    data1c = shape_norm_for_heatmap_incarnato_whole_genome_frag[idx+2]
    data2c = shape_norm_for_heatmap_zhang_invitro_whole_genome_frag[idx+2]
    data3c = shape_norm_for_heatmap_zhang_invivo_whole_genome_frag[idx+2]
    data4c = shape_norm_for_heatmap_pyle_whole_genome_frag[idx+2]
    
    xaxisc = xaxis_whole_genome_frag[idx+2]
    conservationc = conservation_whole_genome_frag[idx+2]
    struct1c = incarnato_struct_track[idx+2]
    struct2c = pyle_struct_track[idx+2]
    struct3c = rnaz_struct_track[idx+2]
    struct4c = zhang_struct_track[idx+2]

    # Now on to making the plot!   
    fig,ax = plt.subplots(11, 1, gridspec_kw={'height_ratios': [.5, 2, 5, 1, .5, 2, 5, 1, .5, 2, 5]},figsize=(120, 60))
    fig.subplots_adjust(wspace=0.50)


    #Tracks
    conserv_track(conservationa, xaxisa, 0)
    struct_track(struct1a,struct2a,struct3a,struct4a, xaxisa, 1)
    shape_track(data1a, data2a, data3a, data4a, xaxisa, 2)  


    ax[3].remove()

    conserv_track(conservationb, xaxisb, 4)
    struct_track(struct1b,struct2b,struct3b,struct4b, xaxisb, 5)
    shape_track(data1b, data2b, data3b, data4b,xaxisb,6)  

    ax[7].remove()
    conserv_track(conservationc, xaxisc, 8)
    struct_track(struct1c,struct2c,struct3c,struct4c, xaxisc, 9)
    shape_track(data1c, data2c, data3c, data4c,xaxisc,10)  
   
    plt.xlabel("Genome position") # X axis label    
    pp.savefig(bbox_inches='tight')
pp.close()